# Modelos de Predicción

In [1]:
# Importaciones necesarias para el correcto funcionamiento de todos lo modelos y demás operaciones con los datos
import pandas as pd 
import numpy as np
import sklearn

In [2]:
result_df = pd.read_csv("Fitabase Data 4.12.16-5.12.16/test_train_data.csv")
print(result_df)

                Id                 Time   HeartRate  Intensity  Calories
0       2022484408  2016-04-12 07:21:00  101.600000          1   3.32064
1       2022484408  2016-04-12 07:22:00   87.888889          1   3.94326
2       2022484408  2016-04-12 07:23:00   58.000000          0   1.34901
3       2022484408  2016-04-12 07:24:00   58.000000          0   1.03770
4       2022484408  2016-04-12 07:25:00   56.777778          0   1.03770
...            ...                  ...         ...        ...       ...
333141  8877689391  2016-05-12 13:55:00   60.666667          0   1.33353
333142  8877689391  2016-05-12 13:56:00   61.875000          0   1.33353
333143  8877689391  2016-05-12 13:57:00   58.142857          0   1.33353
333144  8877689391  2016-05-12 13:58:00   61.200000          0   1.33353
333145  8877689391  2016-05-12 13:59:00   58.000000          0   1.33353

[333146 rows x 5 columns]


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

# Convertir la columna 'Time' a tipo datetime en data_intensities_minutes
result_df['Time'] = pd.to_datetime(result_df['Time'])

# Extraemos características de la columna 'Time'
result_df['Hour'] = result_df['Time'].dt.hour
result_df['Minutes'] = result_df['Time'].dt.minute
result_df['Weekday'] = result_df['Time'].dt.weekday

# Definir las características y la variable objetivo
features = ['Id',  'Hour', 'Minutes', 'Intensity', 'Calories']
target = 'HeartRate'

# Separar las características y la variable objetivo
X = result_df[features]
y = result_df[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11101)

# Crear un transformador para manejar las variables categoricas
categorical_features = ['Id']
numeric_features = ['Hour', 'Minutes', 'Intensity','Calories']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Modelo de Predicción: Red Neuronal

In [4]:
from sklearn.neural_network import MLPRegressor

# Crear el pipeline con preprocesamiento y el modelo MLPRegressor
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', MLPRegressor(random_state=42))])

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Evaluar el modelo
score = pipeline.score(X_test, y_test)
print("R2 score:", score)

R2 score: 0.8407210940159974


In [5]:
best_params = {'regressor__activation': 'tanh', 'regressor__alpha': 0.001, 'regressor__hidden_layer_sizes': (100, 50)}

# Crear el modelo MLPRegressor con los mejores hiperparámetros
mlp = MLPRegressor(random_state=42, max_iter=500, 
                   activation=best_params['regressor__activation'], 
                   alpha=best_params['regressor__alpha'], 
                   hidden_layer_sizes=best_params['regressor__hidden_layer_sizes'])

# Crear el pipeline con preprocesamiento y el modelo MLPRegressor
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', mlp)])

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Evaluar el modelo con los mejores hiperparámetros
score = r2_score(y_test, y_pred)
print("R2 score con mejores hiperparámetros:", score)

R2 score con mejores hiperparámetros: 0.8505155898549964


# Long Short Time Memory RNN

In [ ]:
pip install tensorflow scikit-learn

In [ ]:

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


# Escalar los datos (importante para las redes neuronales)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Remodelar los datos para que sean compatibles con una LSTM
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Crear el modelo LSTM
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(1, len(features))))
model.add(Dense(units=1))  # Capa de salida

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32)

# Hacer predicciones en el conjunto de prueba
predictions = model.predict(X_test_reshaped)

# Calcular el rendimiento del modelo
r2 = r2_score(y_test, predictions)
print(f'R^2 Score: {r2}')

# Modelo de Predicción: KNN Regressor

In [6]:
from sklearn.neighbors import KNeighborsRegressor

# Crear el modelo KNN Regressor
model = KNeighborsRegressor(n_neighbors=5)  # Ajustar el valor de k

# Crear el pipeline que combina el preprocesamiento y el modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
score = pipeline.score(X_test, y_test)

print(f'R2 Score on Test Set: {score}')

R2 Score on Test Set: 0.8357752037486275


# Hiperparámetros KNN

In [8]:

from sklearn.model_selection import GridSearchCV

# Crear una lista de valores para el hiperparámetro 'n_neighbors'
n_neighbors_range = np.arange(1, 21)  # Explorar valores entre 1 y 20

# Crear un diccionario con los hiperparámetros a ajustar
param_grid = {'model__n_neighbors': n_neighbors_range}  # Usa la notación 'model__' para acceder al parámetro dentro del Pipeline

# Crear el modelo KNN Regressor
model = KNeighborsRegressor()

# Crear el pipeline que combina el preprocesamiento y el modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Asume que tienes el preprocessor ya definido
    ('model', model)
])

# Usar GridSearchCV para encontrar la mejor combinación de hiperparámetros
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')  # Validación cruzada con 5 folds
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_

# Crear el modelo KNN Regressor con los mejores hiperparámetros
model = KNeighborsRegressor(**best_params)  # Desempaqueta los parámetros

# Entrenar el modelo con los mejores hiperparámetros
model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
score = model.score(X_test, y_test)
print(f'R2 Score on Test Set (con hiperparámetros ajustados): {score}')

KeyboardInterrupt: 

# Modelo de Predicción: Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline  # Make sure to import Pipeline

# DecisionTreeRegressor Model
dt_regressor = DecisionTreeRegressor(random_state=11101)

# Construir el pipeline con el transformador y el modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', dt_regressor)
])  # Add a closing parenthesis here

# Cross Validation execution
scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
mse_scores = -scores  # Convertir a positivo, ya que cross_val_score devuelve la negación del MSE

# Entrenar el modelo en el conjunto de entrenamiento
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mse_test = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse_test}')

# Matriz de confusión
print("Matriz de Confusión:")
print(np.round(np.array([[mse_test]]), decimals=2))


# Modelo de predicción : SVM - Support Vector Machine Regressor

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Definir las características y la variable objetivo
features = ['Hour', 'Minutes', 'Intensity', 'Calories']
target = 'HeartRate'

# Separar las características y la variable objetivo
X = result_df[features]
y = result_df[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11101)

# Crear el modelo SVM Regressor
model = SVR(kernel='rbf', gamma='auto', epsilon=0.1)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
score = r2_score(y_test, y_pred)
print("R2 score:", score)

# Modelo de predicción: Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Crear el modelo de Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=11101)

# Crear el pipeline que combina el preprocesamiento y el modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
score = pipeline.score(X_test, y_test)

print(f'R2 Score on Test Set: {score}')

# Predicción a futuro

In [ ]:
# Especificar el ID del usuario que deseas seleccionar
usuario_especifico_id = 2022484408

# Verificar si el ID específico está presente en result_df
if usuario_especifico_id in result_df['Id'].values:
    # Filtrar los datos para obtener solo las filas correspondientes al usuario específico
    test_id = result_df[result_df['Id'] == usuario_especifico_id][features]

    # Imprimir los datos de X_test
    print(test_id)22
else:
    print(f"No se encontraron filas para el ID '{usuario_especifico_id}' en result_df.")

pipeline.predict(test_id)